In [1]:
import pandas as pd
from dynaconf import LazySettings
from dynaconf.utils.boxing import DynaBox
from typing import List
import os

In [2]:
config_file = "/home/tiziano/workspaces/fantasAi_football/config/conf.yaml"
config_mode = 'default'

In [3]:
def data_ingestion_basics(params: DynaBox, table: str, duplicate_key_err:str='raise') -> pd.DataFrame:
    import os 
    
    """Performs some basic harmonization steps on a input dataframe.

    Note that after the basic harmonization the columns will have the names
    stated in settings.COLS, and not those appearing in the input data.

    Args:
        table (pd.DataFrame): id of the table to lead, it must appear in
            config file
        duplicate_key_err ('raise' or 'drop'): how to handle duplicated key
        errors. If 'raise' an error will be return in case of duplicated keys,
        if 'drop' all duplicated rows will be dropped from data.

    Raises:
        ValueError: If table registry dataframe keys are missing
        KeyError: If table registry dataframe keys are duplicated

    Returns:
        pd.DataFrame: dataframe in input, harmonized
    """
    # Load parameters
    t_par = params[table]
    t_cols_dict = t_par["COLS"].to_dict()
    keys: List[str] = [t_par["COLS"][c] for c in t_par.KEY]
    cols: List[str] = list(t_cols_dict.values())
    dtype_map = {
        c_name: params["FEATURES"]["DTYPES"][c]
        for c, c_name in t_cols_dict.items()
    }
    name_conversion = {
        c_from: params["FEATURES"][c] for c, c_from in t_cols_dict.items()
    }
    
    path = os.path.join(
        params["PATHS"]["ROOT_FOLDER"], 
        params["PATHS"]["INPUT"]["FOLDER"],
         params["PATHS"]["INPUT"][table]
    )
    data: pd.DataFrame = pd.read_csv(path)

    # Keep only selected columns
    data = data[cols]

    # Ugly but functional call to ensure correct type conversion
    data = data.convert_dtypes().astype(dtype_map, errors='ignore').convert_dtypes()  # type: ignore

    # Ensure that the product registry dataframe keys are valid
    if not data[keys].notna().all(axis=1).all():
        raise ValueError(f"{table} dataframe keys are missing")
    if not data.value_counts(keys).eq(1).all():
        if duplicate_key_err == 'raise':
            raise KeyError(f"{table} keys are duplicated")
        else:
            print(f"WARN: dupliated keys will be removed from {table}")
            data = data.drop_duplicates(subset=keys, keep=False)

    # Sort product product registry by DIVISION,PRODUCT and reset index
    data = data.sort_values(keys).reset_index(drop=True)

    data = data.rename(columns=name_conversion)

    if "FILTER" in params:
        if table in params["FILTER"]:
            for column, values in params["FILTER"].get(table).to_dict().items():
                column_name = params["FEATURES"].get(column)
                data = data.loc[data[column_name].isin(values)]

    return data

In [4]:
params = LazySettings(settings_files=[config_file])
params = params[config_mode]

In [5]:
players = data_ingestion_basics(params, "PLAYERS")
clubs = data_ingestion_basics(params, "CLUBS")
competitions = data_ingestion_basics(params, "COMPETITIONS")
games = data_ingestion_basics(params, "GAMES")
games[params["FEATURES"]["DATE"]] = pd.to_datetime(games[params["FEATURES"]["DATE"]])
#international_competitions_stats = data_ingestion_basics(params, "INTERNATIONAL_COMPETITIONS_STATS")
player_valuations = data_ingestion_basics(params, "PLAYER_VALUATIONS")
players = data_ingestion_basics(params, "PLAYERS")
appearances = data_ingestion_basics(params, "APPEARANCES")
votes_ita = data_ingestion_basics(params, "VOTES_ITA", duplicate_key_err='drop')

WARN: dupliated keys will be removed from VOTES_ITA


## De-normalize data


Find a player appearences for a given year.

In [6]:
# Get appearencs of players in games
# For some reason competition is specified also on appereances table, this
# is not necessary and annoying since it generates suffixed names when merged.
apps_sel = appearances.drop(columns=[params["FEATURES"]["COMPETITION"]])
app_game = apps_sel.merge(games, on=[params["FEATURES"]["GAME"]])

In [7]:
# Get appearencs of players in games
# For some reason competition is specified also on appereances table, this
# is not necessary and annoying since it generates suffixed names when merged.
apps_sel = appearances.drop(columns=[params["FEATURES"]["COMPETITION"]])
app_game = apps_sel.merge(games, on=[params["FEATURES"]["GAME"]])


app_game = app_game.groupby(
    [
        params["FEATURES"]["PLAYER"],
        params["FEATURES"]["SEASON"],
        params["FEATURES"]["COMPETITION"],
        params["FEATURES"]["CLUB"]        
    ],
    as_index=False,
).agg(
    {
        params["FEATURES"]["GOALS"]: "sum",
        params["FEATURES"]["ASSISTS"]: "sum",
        params["FEATURES"]["MINUTES_PLAYED"]: "sum",
        params["FEATURES"]["DATE"]: ["max", "min"],
    }
)

In [8]:
def clean_multiindex_agg(frame: pd.DataFrame) -> pd.DataFrame:
    frame.columns = list(map('||'.join, frame.columns.values))
    frame = frame.rename(
        columns={
            "date||max": "_date_max",
            "date||min": "_date_min"}
            )
    frame.columns = [c.replace("||sum", "") for c in frame.columns]
    frame.columns = [c.replace("||count", "") for c in frame.columns]
    frame.columns = [c.replace("||", "") for c in frame.columns]

    return frame

In [9]:
app_game = clean_multiindex_agg(app_game)

app_game = app_game.rename(
    columns={params["FEATURES"]["GAME"]: params["FEATURES"]["MINUTES_PLAYED"]}
)
app_game["_permanence_week"] = (app_game["_date_max"] - app_game["_date_min"]).dt.days/7

In [10]:
players_team = players.merge(app_game, on=[params["FEATURES"]["PLAYER"]])

Load player value for each season

In [49]:
# Define a start date for a season (player value will be read from this)
players_team["year"] = players_team[params["FEATURES"]["SEASON"]]
players_team["month"] = 9
players_team["day"] = 1
players_team["_season_starts"] = pd.to_datetime(players_team[["year", "month", "day"]])
players_team = players_team.drop(columns=["year", "month", "day"])

In [50]:
# Load player valuation closest to the start of the season
players_vals = players_team.merge(player_valuations, on=[params["FEATURES"]["PLAYER"]])
players_vals["_date_diff"] = (players_vals[params["FEATURES"]["DATE"]] - players_vals["_season_starts"]).dt.days
players_vals = players_vals.loc[players_vals["_date_diff"] < 0]

pv_key = [params["FEATURES"]["PLAYER"], params["FEATURES"]["SEASON"]]
players_vals = players_vals[players_vals.groupby(pv_key)["_date_diff"].transform(max) == players_vals["_date_diff"]]
players_lost = len(players_team) - len(players_vals)
players_vals = players_vals.drop(columns=["_date_diff", "_season_starts", params["FEATURES"]["DATE"]])
print(f"{players_lost}/{len(players_team)} players/seasons has been lost when trying to calculate the value.")

5689/112218 players/seasons has been lost when trying to calculate the value.


In [52]:
# Add informationa about total minutage available in the competition
total_minutes = games.groupby(
    [
        params["FEATURES"]["SEASON"],
        params["FEATURES"]["COMPETITION"],
        params["FEATURES"]["HOME_CLUB"],
    ],
    as_index=False,
).agg({
        params["FEATURES"]["GAME"]: "count",        
        params["FEATURES"]["DATE"]: ["max", "min"],
    })

In [53]:
# Add informationa about total minutage available in the competition
total_minutes = games.groupby(
    [
        params["FEATURES"]["SEASON"],
        params["FEATURES"]["COMPETITION"],
        params["FEATURES"]["HOME_CLUB"],
    ],
    as_index=False,
).agg({
        params["FEATURES"]["GAME"]: "count",        
        params["FEATURES"]["DATE"]: ["max", "min"],
    })

total_minutes = clean_multiindex_agg(total_minutes)

total_minutes["_season_weeks"] = (total_minutes["_date_max"] - total_minutes["_date_min"]).dt.days/7

total_minutes = total_minutes.groupby(
    [params["FEATURES"]["SEASON"], params["FEATURES"]["COMPETITION"]],
    as_index=False,
).mean()
total_minutes[params["FEATURES"]["MINUTES_AVAILABLE"]] = total_minutes[params["FEATURES"]["GAME"]] * 90 * 2


In [56]:
players_vals = players_vals.sort_values(by=pv_key)
players_vals[params["FEATURES"]["VALUE_DELTA"]] = players_vals[params["FEATURES"]["MARKET_VALUE"]].diff()
players_vals[params["FEATURES"]["VALUE_DELTA"]] = players_vals[params["FEATURES"]["VALUE_DELTA"]]/players_vals[params["FEATURES"]["MARKET_VALUE"]]

# Calculate club value
players_vals["_club_value"] = players_vals.groupby(
    [params["FEATURES"]["CLUB"], params["FEATURES"]["SEASON"]]
)[params["FEATURES"]["MARKET_VALUE"]].transform("sum")

# Express club value as ration from media club value of same competition
players_vals["_median_club_value"] = players_vals.groupby(
    [params["FEATURES"]["COMPETITION"], params["FEATURES"]["SEASON"]]
)["_club_value"].transform("median")
players_vals[params["FEATURES"]["CLUB_VALUE_RATIO"]] = (
    players_vals["_club_value"] / players_vals["_median_club_value"]
)

# Express players value as ratio from median player value of same competition
players_vals["_player_median_value"] = players_vals.groupby(
    [params["FEATURES"]["COMPETITION"], params["FEATURES"]["SEASON"]]
)[params["FEATURES"]["MARKET_VALUE"]].transform("median")

players_vals[params["FEATURES"]["MARKET_VALUE_RATIO"]] = (
    players_vals[params["FEATURES"]["MARKET_VALUE"]]
    / players_vals["_player_median_value"]
)


players_onfield = players_vals.merge(total_minutes, on=[params["FEATURES"]["COMPETITION"], params["FEATURES"]["SEASON"]])

players_onfield["_season_played"] = players_onfield["_permanence_week"]/total_minutes["_season_weeks"]
players_onfield["_season_played"] = players_onfield["_season_played"].clip(upper=1)
players_onfield[params["FEATURES"]["ON_FIELD_INDEX"]] = (players_onfield["minutes_played"]/players_onfield[params["FEATURES"]["MINUTES_AVAILABLE"]])*players_onfield["_season_played"]

#players_onfield = players_onfield.drop(columns=["_player_median_value", "_median_club_value", params["FEATURES"]["MINUTES_AVAILABLE"]])

In [59]:
clubs_sel = clubs[[params["FEATURES"]["CLUB"], params["FEATURES"]["CLUB_PRETTY_NAME"]]]
tm_dataset = players_onfield.merge(clubs_sel, on=[params["FEATURES"]["CLUB"]])

In [60]:
path = os.path.join(
    params["PATHS"]["ROOT_FOLDER"],
    f"{params['PATHS']['STAGES']['TM_DATASET']}.pkl"
    )
tm_dataset.to_pickle(path)

In [61]:
# Do some harmonization and save votes ita
votes_ita[params["FEATURES"]["SEASON"]] = votes_ita[params["FEATURES"]["SEASON"]] + 2000
#votes_ita[params["FEATURES"]["TEAM"]].str.replace(params["VOTES_ITA"]["TEAM_TRANSLATOR"])
path = os.path.join(
    params["PATHS"]["ROOT_FOLDER"],
    f"{params['PATHS']['STAGES']['VOTES_ITA']}.pkl"
    )
votes_ita.to_pickle(path)

In [64]:
#tm_dataset.loc[tm_dataset.player_id.isin(['341092', '368482']) & (tm_dataset["season"] == 2020)]
tm_dataset.loc[tm_dataset.player_id.isin(['341092']) & (tm_dataset["season"] == 2020)]

,player_id,name,pretty_name,country_of_citizenship,date_of_birth,position,sub_position,foot,height_in_cm,season,...,_median_club_value,club_value_ratio,_player_median_value,market_value_ratio,game_id,_season_weeks,minutes_available,_season_played,on_field_index,club_pretty_name
3659,341092,federico-chiesa,Federico Chiesa,Italy,25-10-97,Attack,attack - Right Winger,Right,175,2020,...,258053000.0,1.901121,3600000.0,12.0,16.956522,31.683230,3052.173913,NaN,NaN,Ac Florenz
6557,341092,federico-chiesa,Federico Chiesa,Italy,25-10-97,Attack,attack - Right Winger,Right,175,2020,...,1026000000.0,2.255374,10800000.0,4.0,4.121212,15.614719,741.818182,NaN,NaN,Juventus Turin
6648,341092,federico-chiesa,Federico Chiesa,Italy,25-10-97,Attack,attack - Right Winger,Right,175,2020,...,278640000.0,8.304673,4050000.0,10.666667,1.368421,1.629073,246.315789,NaN,NaN,Juventus Turin
6672,341092,federico-chiesa,Federico Chiesa,Italy,25-10-97,Attack,attack - Right Winger,Right,175,2020,...,258053000.0,8.967204,3600000.0,12.0,16.956522,31.683230,3052.173913,NaN,NaN,Juventus Turin
6919,341092,federico-chiesa,Federico Chiesa,Italy,25-10-97,Attack,attack - Right Winger,Right,175,2020,...,2187900000.0,1.057642,22500000.0,1.92,1.000000,0.000000,180.000000,NaN,NaN,Juventus Turin
